# Testing Notebook

### Import libraries

In [22]:
import sys
import os

# Manually specify the path to the 'src' directory
src_path = os.path.abspath(os.path.join(os.getcwd(), '../src'))

# Add the specified path to the system path
sys.path.append(src_path)

# Third-party libraries
import networkx as nx
from qiskit import transpile
from qiskit_ibm_runtime.fake_provider import FakeGuadalupeV2
import mapomatic as mm

# Local application/library imports
import circuit_gen as circuitGen
import max_clique as maxClique
import qubit_mapping as qMap

### Parameters Configuration

In [23]:
num_modules       = 4
module_max_qubits = 4
module_max_gates  = 6
buffer_distance   = 1

### Random Circuit Generation and Mapping Computation

In [ ]:
# Generate random circuit
circuit = circuitGen.RandomCircuit(num_modules, module_max_qubits, module_max_gates)
circuit.gen_random_circuit()

# Ploy dependency graph
circuit.draw_dependency_graph()

In [25]:
# Generate the Quantum Circuit Mapping
q_map = qMap.QubitMapping(circuit, backend=FakeGuadalupeV2(), buffer_distance=buffer_distance)
q_map.generate_qubit_mapping()

In [ ]:
def generate_layouts(module, backend):
    """ Searches for, optimizes and evaluates quantum circuit layouts for a specified backend. """    
    trans_qc = transpile(module, backend, optimization_level=3)
    small_qc = mm.deflate_circuit(trans_qc)
    layouts = mm.matching_layouts(small_qc, backend)
    scores = mm.evaluate_layouts(small_qc, layouts, backend)
    return scores

backend   = q_map.backend
couplings = backend.coupling_map.get_edges()
qc_graph  = nx.from_edgelist(couplings)

# Retrieve the qubits associated with each chosen layout
timestep       = 0
chosen_layouts = []
for vertex in q_map.qubit_mapping[timestep]:
    layouts = generate_layouts(q_map.modules[vertex[0]], backend)
    chosen_layouts.append(layouts[vertex[1]])
# print(f"Qubit associated with each layout: {chosen_layouts}")

# Plot the quantum circuit graph with the chosen layout for each mapped module
colors    = ['blue', 'red', 'green', 'yellow', 'orange', 'purple', 'brown', 'pink']
color_map = []

for node in qc_graph:
    in_layout = False
    for layoutIndex, layout in enumerate(chosen_layouts):
        if node in layout[0]:
            color_map.append(colors[layoutIndex + 1])
            in_layout = True
            
    if not in_layout:
        color_map.append(colors[0])
        
nx.draw(qc_graph, node_color=color_map, with_labels=True)

In [ ]:
# Plot module of the circuit
circuit.modules[1].draw('mpl')

In [ ]:
# Plot complete circuit
circuit.get_circuit().draw('mpl')